In [1]:
%matplotlib inline

import pyxem as px
from diffsims import generators
import tensorflow as tf
import diffpy.structure
from matplotlib import pyplot as plt
import numpy as np
import hyperspy as hs
from tqdm import tqdm
from diffpy.structure import loadStructure
import gc
import os

os.chdir(r'E:\Elisabeth\SPED-phase-mapping')
from ANN import preprocess as pp
from ANN import training_data_functions as tdf

### Import experimental data

In [2]:
signal = hs.io.load(r'Data/datasetA_preprocessed.hspy', lazy=True)

### Load and create structures

In [3]:
os.chdir(r'cif-files')

thetaprime = loadStructure('thetaprime.cif', fmt="cif")
T1 = loadStructure('T1.cif', fmt="cif")
print('Lattice vectors: theta prime: {}, T1: {}.'.format(thetaprime.lattice, T1.lattice))

Lattice vectors: theta prime: Lattice(a=4.04, b=4.04, c=5.8, alpha=90, beta=90, gamma=90), T1: Lattice(a=4.94775, b=4.94775, c=14.145, alpha=90, beta=90, gamma=120).


### Create training set

In [4]:
def simulate(structure, label, DP_scale, weight, num_iterations, euler_angles, HT, relrods, relrods_Al):
    ''''
    Parameters
    --------
    structure : The structure to be simulated. If 'Al', the Al structure will be created inside the function to vary the lattice parameters to account for strain
    label : int - The label of the structure
    DP_scale : float
    weigt : float
    num_iterations : int - The number of simulations per structure
    euler_angles : List holding all three euler angles
    HT : int - The high tension voltage used in the TEM experiments
    relrods : A list holding the lower and upper value for the excitation error for each precipitate
    relrods_Al : A list holding the lower and upper value for the excitation error for Al
    
    Returns:
    patterns : A list with length num_iterations holding the simulated patterns of structure
    labels : A list with length num_iterations holding the labels of structure
    '''
    central_beam_mask = px.utils.expt_utils.circular_mask((128,128), 11)
    ediff = generators.diffraction_generator.DiffractionGenerator(HT, shape_factor_model='lorentzian')
    
    #Ensure that we have the same amount of simulated patterns per zone axis of precipitates:
    if structure == structure_thetaprime_100:
        num_iterations = int(num_iterations/2)

    elif structure == structure_T1:
        num_iterations = int(num_iterations/2)
    
    labels = np.zeros((num_iterations), dtype='int')
    patterns = np.zeros((num_iterations, 128, 128), dtype='float32')
    
    for i in range(num_iterations):
        # Simulation parameters:
        phi_A = 3 * (np.random.rand() - 0.5) # Slightly change the in-plane rotation
        phi_T = 4*(np.random.rand() - 0.5) # Slightly change the sample tilt
        rand_eulerx =4*(np.random.rand() - 0.5)
        
        weight_effective = (weight[1] - weight[0]) * np.random.random_sample() + weight[0] # Varies the weighting between Al and precipitate
        sim = 1.9 + (np.random.rand()-0.5)*0.6 # Varies the size of the diffracted reflections
        relrod = (relrods[0] - relrods[1]) * np.random.random_sample() + relrods[1] # Varies the excitation error for the precipitates
        relrod_Al = (relrods_Al[0] - relrods_Al[1]) * np.random.random_sample() + relrods_Al[1] # Varies the excitation error for Al
        
        # Account for all the orientation relationships of the precipitates:
        if structure != 'Al':
            diffraction_Al = ediff.calculate_ed_data(tdf.structure_Al(), reciprocal_radius=reciprocal_radius, max_excitation_error=relrod_Al, with_direct_beam=False,
                                             rotation=(phi_A, rand_eulerx, phi_T))
            diffraction_Al.calibration = DP_scale
            pattern_Al = diffraction_Al.get_diffraction_pattern((128, 128), sigma=sim)
            diffraction_preci = ediff.calculate_ed_data(structure, reciprocal_radius=reciprocal_radius, max_excitation_error=relrod, with_direct_beam=False,
                                                  rotation=(phi_A + euler_angles[0], rand_eulerx + euler_angles[1], phi_T + euler_angles[2]))
            diffraction_preci.calibration = DP_scale
            pattern_preci = diffraction_preci.get_diffraction_pattern((128, 128), sigma=sim)
            pattern_preci = np.average(np.array([pattern_preci, weight_effective*pattern_Al]),axis=0)
        else:
            diffraction_Al = ediff.calculate_ed_data(tdf.structure_Al(), reciprocal_radius=reciprocal_radius, max_excitation_error=relrod_Al, with_direct_beam=False,
                                             rotation=(phi_A, rand_eulerx, phi_T)) 
            diffraction_Al.calibration = DP_scale
            pattern_Al = diffraction_Al.get_diffraction_pattern((128,128), sigma=sim)
            pattern_preci = pattern_Al
        pattern_preci = pp.normalize(pattern_preci, only_one_image=True)
        pattern_preci = tdf.poisson_noise(pattern_preci)
        pattern_preci = tdf.gaussian_noise(pattern_preci)
        pattern_preci = tdf.radial_noise(pattern_preci, plot=False)
        pattern_preci = pp.log_shift(pattern_preci, shift=((0.10 - 0.01) * np.random.random_sample() + 0.01)) #0.15 - 0.01
        pattern_preci = pp.normalize(pattern_preci, only_one_image=True)
        pattern_preci = pattern_preci * ~central_beam_mask
        pattern_preci = pattern_preci.astype('float32')
        patterns[i] = pattern_preci
        labels[i] = label
    return patterns, labels

In [5]:
structure_thetaprime_100 = thetaprime.copy()
structure_thetaprime_001 = thetaprime.copy()
structure_T1 = T1.copy()

# Calibrate reciprocal space
target_pattern_dimensions_pixels = signal.axes_manager[2].size
half_size = target_pattern_dimensions_pixels//2
DP_scale = signal.axes_manager[2].scale
reciprocal_radius = DP_scale*(half_size-1)

print('Starting simulations..')

structures = ['Al', structure_thetaprime_100, structure_thetaprime_100, structure_thetaprime_001, structure_T1, structure_T1]
structure_labels = np.array([0, 1, 1, 2, 3, 3], dtype='int')
num_iter = 10000

# Choose simulations parameters
euler_angles = [[0, 0, 0], [0, 90, 0], [90, 90, 0], [0, 0, 0], [45, 54.7, 60], [-45, 54.7, 60]]
weight = [[1,1], [1 ,10], [1, 10],  [1., 50], [1., 550], [1., 550]]
relrods = [[0.11, 0.04], [0.3, 0.02], [0.3, 0.02], [0.15, 0.02], [0.15, 0.01],  [0.15, 0.01]]
relrod_Al = [0.15, 0.02]

for i in range(len(structures)):
    print('Simulation bulk nr ', i)
    patterns, labels = simulate(structure=structures[i], label=structure_labels[i], DP_scale=DP_scale, weight = weight[i], 
                                num_iterations=num_iter, euler_angles=euler_angles[i], HT=200, 
                                relrods = relrods[i], relrods_Al=relrod_Al)
    if i == 0:
        all_patterns, all_labels = patterns.copy(), labels.copy()
    else:
        all_patterns = np.concatenate((all_patterns, patterns))
        all_labels = np.concatenate((all_labels, labels))

Starting simulations..
Simulation bulk nr  0
Simulation bulk nr  1
Simulation bulk nr  2
Simulation bulk nr  3
Simulation bulk nr  4
Simulation bulk nr  5


### Split into training, validation and test set

In [6]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

num_patterns = len(all_patterns)
all_labels_shuffled = []
all_patterns_shuffled = []

all_labels_shuffled, all_patterns_shuffled = shuffle(all_labels, all_patterns)

train_size = 0.8

train_data, rem_data, train_label, rem_label = train_test_split(all_patterns_shuffled, all_labels_shuffled,
                                                               train_size=train_size)

test_size = 0.5
test_data, val_data, test_label, val_label = train_test_split(rem_data, rem_label, train_size=test_size)

test_data, val_data, train_data, test_label, val_label, train_label = np.asarray(test_data), np.asarray(val_data), np.asarray(train_data), np.asarray(test_label), np.asarray(val_label), np.asarray(train_label)


print('Total number of patterns: {}. Patterns in train set: {}. Patterns in validation set: {}. Patterns in test set: {}'.format(all_patterns.shape[0],train_data.shape[0], val_data.shape[0], test_data.shape[0]))

Total number of patterns: 40000. Patterns in train set: 32000. Patterns in validation set: 4000. Patterns in test set: 4000


## Train neural network

In [7]:
initializer =  tf.compat.v1.initializers.glorot_uniform()
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(1800, activation='tanh', kernel_initializer=initializer),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='Adamax',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)

reduce_lr =  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.001)

In [9]:
model.fit(train_data, train_label, epochs=30, validation_data=(val_data, val_label), callbacks=[earlystopping, reduce_lr])
model.evaluate(test_data, test_label)

Epoch 1/30
1000/1000 [==============================] - 123s 122ms/step - loss: 0.7519 - accuracy: 0.6580 - val_loss: 0.5038 - val_accuracy: 0.7893 - lr: 0.0010
Epoch 2/30
1000/1000 [==============================] - 122s 122ms/step - loss: 0.4299 - accuracy: 0.8145 - val_loss: 0.3051 - val_accuracy: 0.8798 - lr: 0.0010
Epoch 3/30
1000/1000 [==============================] - 124s 124ms/step - loss: 0.2939 - accuracy: 0.8859 - val_loss: 0.2374 - val_accuracy: 0.8942 - lr: 0.0010
Epoch 4/30
1000/1000 [==============================] - 132s 132ms/step - loss: 0.2223 - accuracy: 0.9177 - val_loss: 0.1545 - val_accuracy: 0.9675 - lr: 0.0010
Epoch 5/30
1000/1000 [==============================] - 130s 130ms/step - loss: 0.1536 - accuracy: 0.9511 - val_loss: 0.1866 - val_accuracy: 0.9325 - lr: 0.0010
Epoch 6/30
1000/1000 [==============================] - 133s 133ms/step - loss: 0.1414 - accuracy: 0.9507 - val_loss: 0.1605 - val_accuracy: 0.9333 - lr: 0.0010
Epoch 7/30
1000/1000 [============

[0.04024282097816467, 0.9904999732971191]

In [10]:
directory = r'E:\Elisabeth\SPED-phase-mapping\Data\Outputs'.replace('\\','\\\\')
model.save(directory + '/ANN_trained_model')